## Client
The client controls the experiment. The client doesn't need the PYNQ device drivers, so it can be a Linux or Windows PC (you should be able to install the QICK libraries as usual, the PYNQ library will be skipped on systems that don't support it).

The nameserver and server must already be running (see the previous notebooks in this directory).

The cells below contain the initialization code that replace the typical initialization steps of:
```
from qick import QickSoc
soc = QickSoc()
soccfg = soc
```

**"localhost" is only correct for this proof of concept, where the nameserver is running in the same place as the client. In actual use, your client will be some PC, and the nameserver might be running on the QICK board or a third PC. Either way, you need to use the nameserver's IP or a hostname that resolves to it.**

In [1]:
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

ns_host = "localhost"
ns_port = 8888
proxy_name = "myqick"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)

Let's see what is registered on the nameserver:

In [2]:
# print the nameserver entries: you should see the QickSoc proxy
for k,v in ns.list().items():
    print(k,v)

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8888
myqick PYRO:obj_2d0d1ae01ad847229027ddb66b3e0181@131.225.82.122:39409


Now, connect to the server. The Proxy object replaces the QickSoc in your code. There's one additional step: you need to create a QickConfig object that contains all the information necessary to compile programs.

In the demo notebooks both `soc` and `soccfg` point to the same QickSoc object; when using Pyro, `soc` is a Proxy object and `soccfg` is a QickConfig object.

In [3]:
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	1:	axis_signal_gen_v6 - tProc output 2, envelope memory 65536 samples
		DAC tile 2, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	2:	axis_signal_gen_v6 - tProc output 3, envelope memory 65536 samples
		DAC tile 2, ch 2, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	3:	axis_signal_gen_v6 - tProc output 4, envelope memory 65536 samples
		DAC tile 2, ch 3, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	4:	axis_signal_gen_v6 - tProc output 5, envelope memory 65536 samples
		DAC tile 3, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	5:	axis_signal_gen_v6 - tProc output 6, envelope memory 65536 samples
		DAC tile 3, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	6:	axis_signal_gen_v6 - tProc out

For your convenience, there's a function that encapsulates the code above. You could just put this in your code.

In [4]:
from qick.pyro import make_proxy
soc, soccfg = make_proxy(ns_host="localhost", ns_port=8888, proxy_name="myqick")
print(soccfg)

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8888
myqick PYRO:obj_2d0d1ae01ad847229027ddb66b3e0181@131.225.82.122:39409

QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	1:	axis_signal_gen_v6 - tProc output 2, envelope memory 65536 samples
		DAC tile 2, ch 1, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	2:	axis_signal_gen_v6 - tProc output 3, envelope memory 65536 samples
		DAC tile 2, ch 2, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	3:	axis_signal_gen_v6 - tProc output 4, envelope memory 65536 samples
		DAC tile 2, ch 3, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	4:	axis_signal_gen_v6 - tProc output 5, envelope memory 65536 samples
		DAC tile 3, ch 0, 32-bit DDS, fabric=430.080 MHz, fs=6881.280 MHz
	5:	axis_signal_gen_v6 - tProc output 6, envelope mem

In [5]:
help(make_proxy)

Help on function make_proxy in module qick_pyro:

make_proxy(ns_host, ns_port='8888', proxy_name='myqick')
    Connects to a QickSoc proxy server.
    
    Parameters
    ----------
    ns_host : str
        hostname or IP address of the nameserver
        if the nameserver is running on the same PC you are running make_proxy() on, "localhost" is fine
    ns_port : int
        the port number you used when starting the nameserver
    proxy_name : str
        name for the QickSoc proxy you used when running start_server()
    
    Returns
    -------
    Proxy
        proxy to QickSoc - this is usually called "soc" in demos
    QickConfig
        config object - this is usually called "soccfg" in demos

